In [8]:
# Imports
from strategy.actions import random_action
from data.fetch_data import fetch_data
from data.process_data import calculate_returns
from visualizations.plot_results import plot_net_worth, plot_cash, plot_portfolio_value, plot_rewards
from simulations.run_simulation import run_simulation

import numpy as np
import pandas as pd

In [31]:
tickers = ['AAPL', 'KO', 'IBM']
start_date = "2010-01-01"
end_date = "2017-12-31"

data = fetch_data(tickers, start_date, end_date)
rets = calculate_returns(data)
data

[*********************100%%**********************]  3 of 3 completed


Ticker,AAPL,IBM,KO
Date,,,
2010-01-01,6.351058,73.703934,17.938570
2010-01-08,6.316374,75.274139,18.238665
2010-01-15,6.275355,73.390991,17.357544
2010-01-22,6.010552,70.404167,17.296885
2010-01-29,5.792195,69.977478,17.057447
...,...,...,...
2017-12-01,39.876526,107.056381,37.264786
2017-12-08,40.559509,107.356133,37.468292
2017-12-15,41.216583,105.613365,37.118263


In [55]:
def discretize_price(price, n_states, min_price=0, max_price=40):
    return np.clip(np.floor((price - min_price) / (max_price - min_price) * n_states), 0, n_states - 1).astype(int)

In [85]:
np.random.seed(123)

mapper = {'AAPL': [rets['AAPL'].values, data['AAPL'].values[0], 0, 75], 
            'KO': [rets['KO'].values, data['KO'].values[0], 0, 75],
            'IBM': [rets['IBM'].values, data['IBM'].values[0], 10, 1150]}

initial_price_map = {'AAPL': data['AAPL'].values[0], 
            'KO': data['KO'].values[0],
            'IBM': data['IBM'].values[0]}

paths = {'AAPL': [], 'KO': [], 'IBM': []}
raw_paths = {'AAPL': [], 'KO': [], 'IBM': []}

for tick in tickers:
    emp_dist = mapper[tick][0]

    for i in range(10000):
        ret_path = np.random.choice(emp_dist, size=len(emp_dist), replace=True)
        raw_path = mapper[tick][1] * (1 + ret_path).cumprod()
        raw_paths[tick].append(raw_path)
        paths[tick].append(discretize_price(raw_path, n_states=10, min_price=mapper[tick][2] , max_price=mapper[tick][3]))


In [90]:
# Initialize the Q-table
Q_table = np.zeros((64, 27, 27)) 
Q_table

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [96]:
def map_ret_state_to_real(ret_state):
    real = ((ret_state[0] - 1) * 16) + ((ret_state[1] - 1) * 4) + (ret_state[2] - 1)
    return real

def map_pos_state_to_real(pos_state):
    real = ((pos_state[0] - 1) * 9) + ((pos_state[1] - 1) * 3) + (pos_state[2] - 1)
    return real

def map_real_to_pos_state(real_number):
    # Convert the real number to base-3 (get three digits)
    d1 = real_number // 9   # Corresponds to the first die
    d2 = (real_number % 9) // 3   # Corresponds to the second die
    d3 = real_number % 3   # Corresponds to the third die

    # Map base-3 digits (0, 1, 2) back to dice values (1, 2, 3)
    return np.array([d1 + 1, d2 + 1, d3 + 1])


In [98]:
map_real_to_pos_state(0)

array([1, 1, 1])

In [ ]:
n_episodes = 10000
n_steps =
alpha = 0.1
epsilon = 0.1
gamma = 1
Q_table = np.zeros((64, 27, 27))


for episode in range(n_episodes):
    cash = 100
    # Generate a new stock price series for each episode
    stock_prices = generate_stock_price_series(n_steps)
    stock_returns = 

    raw_holdings = np.array([0, 0, 0])
    holdings = raw_holdings + 2
    returns = np.random.choice([1, 2, 3, 4], 3)
    return_state = map_ret_state_to_real(np.random.choice([1, 2, 3, 4], 3))
    holdings_state = map_pos_state_to_real(holdings)
    
    for t in range(n_steps - 1):
        # Choose action using epsilon-greedy policy
        if np.random.rand() < epsilon:
            action = np.random.choice([-1, 0, 1], 3)
        else:
            action = np.argmax(Q_table[return_state, holdings_state])
            action = map_real_to_pos_state(action) - 2
        
        next_state = discretize_rets(stock_prices[t + 1], n_states)
        reward = 0
        
        for i in range(3):
            if action[i] == 1:  # Buy
                if cash - stock_prices[i] > 0:  # Can only buy if have enough cash
                    reward -= stock_prices[i]
            elif action[i] == 0:
                pass

            elif action[i] == -1:  # Sell
                reward += stock_prices[i]
                holding = 0
        
        # Q-learning update
        Q_table[state, holding, action] += alpha * (reward + gamma * np.max(Q_table[next_state, holding]) - Q_table[state, holding, action])
        
        # Transition to next state
        state = next_state


In [ ]:
raw_paths['AAPL'][0]